In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
import os

# load .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = OpenAI(api_key=openai_api_key)

In [4]:
name = llm("I want to open a restaurant for Salvadorian Food. Suggest a fancy name for this.")
print(name)



"Cocina Salvadoreña" or "El Sabor de El Salvador"


In [4]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate( 
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggent a fency name for this.")
prompt_template_name.format(cuisine="Salvadorian")


'I want to open a restaurant for Salvadorian food. Suggent a fency name for this.'

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Salvadorian")

C:\Users\ericr\OneDrive\Desktop\Projects\Data Science\LLM Projects\LangChain-Resturant-Project\openai-env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\ericr\OneDrive\Desktop\Projects\Data Science\LLM Projects\LangChain-Resturant-Project\openai-env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'\n\n"El Sabor de El Salvador" (The Flavor of El Salvador) '

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("I want to open a restaurant for {cuisine} food. Suggent a fency name for this.")
chain = prompt | llm | StrOutputParser()


In [9]:
chain.invoke({"cuisine": "Salvadorian"})

'\n"El Sabor de El Salvador" (The Flavor of El Salvador)'

In [10]:
menu_prompt = ChatPromptTemplate.from_template("Suggest some menu items for {restaurant_name}. Return it as a comma seperated list.")
composed_chain = {"restaurant_name": chain} | menu_prompt | llm | StrOutputParser()

In [11]:
composed_chain.invoke({"cuisine": "Salvadorian"})

'\n\n1. Pupusas\n2. Yuca frita\n3. Sopa de res\n4. Pollo encebollado\n5. Plátanos fritos\n6. Tamal de elote\n7. Panes con pollo\n8. Frijoles refritos\n9. Chicharrón con yuca\n10. Tamales de cerdo\n11. Enchiladas salvadoreñas\n12. Tostadas de carne asada\n13. Arroz con pollo\n14. Empanadas de plátano\n15. Quesadillas de loroco\n16. Sopa de gallina\n17. Chilate\n18. Pescado frito\n19. Churros\n20. Horchata. '

In [13]:
## Note this method may interfere with operations like streaming
composed_chain_with_lambda = (
    chain
    | (lambda input: {"restaurant_name": input})
    | menu_prompt
    | llm
    | StrOutputParser()
)
composed_chain_with_lambda.invoke({"cuisine": "Salvadorian"})

'\n\nPupusas, Yuca frita, Pollo encebollado, Sopa de res, Plátanos fritos, Tamales, Quesadilla salvadoreña, Empanadas, Horchata, Tres leches cake'

In [14]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"restaurant_name": chain})
    .pipe(menu_prompt)
    .pipe(llm)
    .pipe(StrOutputParser()))

composed_chain_with_pipe.invoke({"cuisine": "Salvadorian"})

'\n\nPupusas, Pollo encebollado, Sopa de res, Plátanos fritos, Yuca con chicharrón, Carne asada, Tamales, Empanadas de plátano, Ensalada de repollo, Horchata.'

In [19]:
print(chain)

first=ChatPromptTemplate(input_variables=['cuisine'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['cuisine'], template='I want to open a restaurant for {cuisine} food. Suggent a fency name for this.'))]) middle=[OpenAI(client=<openai.resources.completions.Completions object at 0x000002369B92DA00>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000002369DEC2F70>, openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()


In [16]:
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [17]:
print(name_chain)

prompt=PromptTemplate(input_variables=['cuisine'], template='I want to open a restaurant for {cuisine} food. Suggent a fency name for this.') llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000002369B92DA00>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000002369DEC2F70>, openai_api_key=SecretStr('**********'), openai_proxy='') output_key='restaurant_name'


In [20]:
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}." )

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [23]:
from langchain.chains import SequentialChain

chain_generator = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items'])

chain_generator({'cuisine':'Salvadorian'})

C:\Users\ericr\OneDrive\Desktop\Projects\Data Science\LLM Projects\LangChain-Resturant-Project\openai-env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Salvadorian',
 'restaurant_name': '\n\n"El Sabor Salvadoreño" (The Salvadorian Flavor)',
 'menu_items': '\n\n1. Pupusas (thick corn tortillas stuffed with cheese, beans, and/or meat)\n2. Sopa de Res (beef soup with vegetables)\n3. Pollo encebollado (chicken in onion sauce)\n4. Tamal de Elote (corn tamale)\n5. Yuca frita (fried cassava)\n6. Plátanos fritos (fried plantains)\n7. Carne Asada (grilled steak)\n8. Ensalada de Repollo (cabbage salad)\n9. Arroz con camarones (rice with shrimp)\n10. Chorizo con queso (sausage and cheese appetizer)\n11. Tamales Salvadoreños (traditional Salvadorian tamales)\n12. Chicharrón con yuca (fried pork belly with cassava)\n13. Sopa de Pata (beef tripe soup)\n14. Atol de Elote (creamy corn drink)\n15. Empanadas de leche (sweet milk turnovers).'}

In [27]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

complete_chain = ({
    "cuisine": itemgetter("cuisine"),
    "restaurant_name": chain}
    | RunnablePassthrough.assign(d=composed_chain))
complete_chain.invoke({"cuisine":"Salvadorian"})

{'cuisine': 'Salvadorian',
 'restaurant_name': '\n\n"El Sabor de El Salvador"',
 'd': '\n\n1. Pupusas: Traditional corn tortillas stuffed with cheese, beans, or meat\n2. Yuca frita: Fried yuca served with a tangy curtido (pickled cabbage) slaw\n3. Sopa de res: Hearty beef and vegetable soup  \n4. Pollo encebollado: Chicken cooked in a savory onion and tomato sauce  \n5. Plátanos fritos: Fried plantains served as a side dish or snack  \n6. Carne asada: Grilled marinated steak served with rice and beans  \n7. Tamales: Corn husk-wrapped pockets of corn dough filled with meat, vegetables, and sauce  \n8. Chorizo con huevos: Scrambled eggs with spicy chorizo sausage  \n9. Ensalada de pupusa: A salad made with shredded pupusas, cabbage, and tomatoes  \n10. Arroz con leche: Sweet rice pudding topped with cinnamon.'}